In [1]:
import json
import pandas as pd
from coinmarketcap import Market
# function that checks for NaN in a pandas df
def check_nan(df):
    count = 0
    for i,j in enumerate(df.isnull().sum()):
        if j != 0:
            print(df.columns[i], "contains {} NaN".format(j))
            count += 1
    if count == 0:
        print("no NaN")


coinmarketcap = Market()
# the output of below call is byte so we need to convert it to str
a = coinmarketcap.ticker()
# conversion bytecode to str
json_thing = a.decode()
json_thing = json.loads(json_thing)
df = pd.DataFrame(json_thing)
df.last_updated = pd.to_datetime(df.last_updated, unit='s')
del df['available_supply']
del df['id']
del df['percent_change_1h']
del df['percent_change_7d']
del df['name']
del df['total_supply']

df = df[['24h_volume_usd', 'symbol', 'last_updated', 'market_cap_usd',
       'percent_change_24h', 'price_btc', 'price_usd', 'rank']]
check_nan(df)
df.dropna(inplace=True)
check_nan(df)
# df.sort_values('24h_volume_usd', ascending=False, inplace=True)
json_thing

24h_volume_usd contains 48 NaN
last_updated contains 2 NaN
market_cap_usd contains 191 NaN
percent_change_24h contains 38 NaN
price_btc contains 2 NaN
price_usd contains 2 NaN
no NaN


[{'24h_volume_usd': '1903140000.0',
  'available_supply': '16389212.0',
  'id': 'bitcoin',
  'last_updated': '1497384550',
  'market_cap_usd': '44800566570.0',
  'name': 'Bitcoin',
  'percent_change_1h': '-0.45',
  'percent_change_24h': '5.15',
  'percent_change_7d': '-3.33',
  'price_btc': '1.0',
  'price_usd': '2733.54',
  'rank': '1',
  'symbol': 'BTC',
  'total_supply': '16389212.0'},
 {'24h_volume_usd': '1886770000.0',
  'available_supply': '92456682.0',
  'id': 'ethereum',
  'last_updated': '1497384567',
  'market_cap_usd': '36993675295.0',
  'name': 'Ethereum',
  'percent_change_1h': '1.84',
  'percent_change_24h': '9.81',
  'percent_change_7d': '54.41',
  'price_btc': '0.147169',
  'price_usd': '400.119',
  'rank': '2',
  'symbol': 'ETH',
  'total_supply': '92456682.0'},
 {'24h_volume_usd': '77977900.0',
  'available_supply': '38326381283.0',
  'id': 'ripple',
  'last_updated': '1497384543',
  'market_cap_usd': '9901812236.0',
  'name': 'Ripple',
  'percent_change_1h': '-0.03',

In [140]:
# limit = 2000 <> rows 2000
# from 2014-10-11 03:00:00
# to 2015-01-02 10:00:00

print(r.json()['TimeFrom'])
print(r.json()['TimeTo'])

1413021600
1420192800


In [179]:
import datetime
print("Date From",datetime.datetime.fromtimestamp(r.json()['TimeFrom']))
print("Date To", datetime.datetime.fromtimestamp(r.json()['TimeTo']))

date_from  = datetime.datetime(2015, 1, 2, 10, 00, 00).timestamp()
date_from

Date From 2014-10-11 04:00:00
Date To 2015-01-02 10:00:00


1420192800.0

In [170]:
datetime.datetime.fromtimestamp(1420196399)

datetime.datetime(2015, 1, 2, 10, 59, 59)

In [268]:
import datetime
import time
dt = parser.parse("2014-10-11 04:00:00")
time.mktime(dt.timetuple())

1412982000.0

In [160]:
from dateutil import parser
dt = parser.parse("2014-10-11 04:00:00")

### Historical data extraction with help of cryptocompare.com API
https://www.cryptocompare.com/api/#introduction

In [28]:
##### Hourly data from cryprocompare

import pandas as pd
import datetime
import requests
from IPython.display import display

# generic url for data extraction from cryptocompare.com's API
url = 'https://min-api.cryptocompare.com/data/{0}?fsym={1}&tsym={2}&limit={3}&toTs={4}&e={5}'

# parameters for the url
# data resolution {0}
hourly_data = "histohour"
minute_data = "histominute"
# currency from {1}
curr_from = "BTC"
# currency to {2}
curr_to = "EUR"

############################################################
# limit of hours with maximum 2000 (1992 is 83 days) {3}
# calculating and converting time for data extraction {4}
date_from_datetime  = datetime.datetime(2017, 6, 1, 0, 0)
print(date_from_datetime)

# enter date_to that is <= than 83 days from date_from for hourly data or <= 83 hr if minute data
date_to_datetime = datetime.datetime(2017, 6, 6, 0, 0)
print(date_to_datetime)

# checking that the entered dates have no more than 2000 hr between them
delta_in_hours = int(((date_to_datetime - date_from_datetime).total_seconds() / 3600))
print("There are {0} hrs or {1:.2f} days between the dates {2} and {3}".format(delta_in_hours, delta_in_hours / 24, \
      date_from_datetime, date_to_datetime))
limit_hours = 2000
assert (delta_in_hours < limit_hours), \
"There must be <= {0} hrs between dates, but the current value is {1}. Change the dates interval"\
.format(limit_hours, delta_in_hours)
    
# calculating the the date_to by adding 1999 hr to the date_from
# date_to = (date_from_datetime + datetime.timedelta(hours=2000-1))
date_to_timestamp = date_to_datetime.timestamp()
print("The Date To timestamp is {}".format(date_to_timestamp))
############################################################

# coin exchange name, if not used, then CCAGG  {5}
exchange = "Kraken"#"&e=Kraken" (default exchange is CCCAGG)



r = requests.get(url.format(minute_data, curr_from, curr_to, delta_in_hours, date_to_timestamp, exchange))

df = pd.DataFrame(r.json()['Data'])
df.time = pd.to_datetime(df.time, unit='s')
if exchange == "": exchange = "CCCAGG"
print("Prices from the exchange: {}".format(exchange))

print("##########################")
df.set_index(df['time'], drop=True, inplace=True)
del df['time']
display(df.head(5))
display(df.tail(5))

2017-06-01 00:00:00
2017-06-06 00:00:00
There are 120 hrs or 5.00 days between the dates 2017-06-01 00:00:00 and 2017-06-06 00:00:00
The Date To timestamp is 1496703600.0
Prices from the exchange: Kraken
##########################


,close,high,low,open,volumefrom,volumeto
time,,,,,,
2017-06-05 21:00:00,2358.67,2359.02,2355.54,2355.96,2.3400,5502.62
2017-06-05 21:01:00,2359.02,2359.02,2355.89,2358.67,1.8700,4419.74
2017-06-05 21:02:00,2359.02,2359.02,2355.96,2359.02,0.3479,820.38
2017-06-05 21:03:00,2359.02,2359.02,2359.01,2359.02,2.1300,5014.99
2017-06-05 21:04:00,2359.02,2359.02,2359.01,2359.02,0.5619,1325.47


,close,high,low,open,volumefrom,volumeto
time,,,,,,
2017-06-05 22:56:00,2376.66,2379.00,2376.66,2376.75,7.35,17470.81
2017-06-05 22:57:00,2374.88,2376.67,2368.05,2376.66,16.48,39097.93
2017-06-05 22:58:00,2373.16,2374.88,2369.00,2374.88,5.78,13719.27
2017-06-05 22:59:00,2368.39,2373.16,2368.39,2373.16,10.43,24731.78
2017-06-05 23:00:00,2371.03,2371.03,2366.26,2368.39,5.29,12533.87


In [20]:
from flask import Flask
app = Flask (__name__)

@app.route("/")
def hello():
    return "hello me!"

if __name__ == "__main__":
    app.run()


 * Running on http://127.0.0.1:5000/ (Press CTRL+C to quit)
127.0.0.1 - - [04/Jun/2017 20:25:06] "GET / HTTP/1.1" 200 -


In [24]:
l1

[2527.42, 2527.42, 2528.92, 2528.92, 2529.12, 2529.12, 2529.12, 2528.8]

In [9]:
import rethinkdb as r
import pandas as pd
import json
r.connect( "localhost", 28015).repl()

In [8]:
authors = r.db("test").table("authors").run()
authors

<rethinkdb.net.DefaultCursor object at 0x7f39e7ded7b8 (done streaming):
 [{'posts': [{'title': 'Civil rights', 'content': "There are some words I've known since..."}], 'name': 'Jean-Luc Picard', 'id': '56d53d54-7ef4-41b7-8846-7619eefae777', 'tv_show': 'Star Trek TNG'}, {'posts': [{'title': 'Decommissioning speech', 'content': 'The Cylon War is long over...'}, {'title': 'We are at war', 'content': 'Moments ago, this ship received...'}, {'title': 'The new Earth', 'content': 'The discoveries of the past few days...'}], 'name': 'William Adama', 'id': '68ce53bd-9e13-4fc0-96f9-923cc7b752f9', 'tv_show': 'Battlestar Galactica'}, {'posts': [{'title': 'The oath of office', 'content': 'I, Laura Roslin, ...'}, {'title': 'They look like us', 'content': 'The Cylons have the ability...'}], 'name': 'Laura Roslin', 'id': 'dfdc8db5-d1fd-4ea9-ad81-a6b88149b463', 'tv_show': 'Battlestar Galactica'}]>

In [12]:
for i in authors:
    print(i.json())

In [367]:
#r.db_create("COIN_PRICES").run()
r.db("COIN_PRICES").table_create("BTCUSD").run()
r.db("COIN_PRICES").table("BTCUSD").insert({'USD': 2510.01}).run()

{'config_changes': [{'new_val': {'db': 'COIN_PRICES',
    'durability': 'hard',
    'id': '9fe39dc0-431f-4603-b9ec-63ee4e8223e7',
    'indexes': [],
    'name': 'BTCUSD_HOUR',
    'primary_key': 'id',
    'shards': [{'nonvoting_replicas': [],
      'primary_replica': 'yol_desktop_jt4',
      'replicas': ['yol_desktop_jt4']}],
    'write_acks': 'majority'},
   'old_val': None}],
 'tables_created': 1}

In [289]:
import datetime
datetime.datetime.fromtimestamp(1420196399)

import datetime
import time
dt = parser.parse("2014-10-11 04:00:00")
time.mktime(dt.timetuple())

tmp = pd.to_datetime(1496839680, unit="s")
tmp.to_pydatetime()


Timestamp('2017-06-07 12:48:00')

In [388]:
import time
import datetime
import requests
import rethinkdb as r
import pandas as pd

r.connect( "localhost", 28015).repl()

url = 'https://min-api.cryptocompare.com/data/histominute?fsym=BTC&tsym=USD&limit=10&aggregate=0&e=Kraken'
def data_extract(url):
    l1 = list()
    while True:
        url_output = requests.get(url).json()['Data']
        url_output[0]['time'] = datetime.datetime.isoformat(datetime.datetime.fromtimestamp(url_output[0]['time']))
        url_output[1]['time'] = datetime.datetime.isoformat(datetime.datetime.fromtimestamp(url_output[1]['time']))
        #df = pd.DataFrame(url_output)
        #df.time = pd.to_datetime(df.time, unit='s')

        

        #display(df)
        r.db("COIN_PRICES").table("BTCUSD").insert(url_output).run()
        time.sleep(60)
        

data_extract(url)  

KeyboardInterrupt: 

In [364]:
from IPython.display import display
import numpy as np
np.set_printoptions(suppress=True)
from_db = r.db("COIN_PRICES").table("BTCUSD").run()
df_from_db = pd.DataFrame()
for i in from_db:

    df_from_db = df_from_db.append(pd.io.json.json_normalize(i))

#df_from_db.time = pd.to_datetime(df_from_db.time, unit='s')
# df_from_db.set_index(df_from_db.time, inplace=True)
# df_from_db.drop(['time'],inplace=True)
df_from_db.sort(columns='time').tail()

/home/yol/miniconda3/envs/tensorflow/lib/python3.5/site-packages/ipykernel_launcher.py:13: FutureWarning: sort(columns=....) is deprecated, use sort_values(by=.....)
  del sys.path[0]


,close,high,id,low,open,time,volumefrom,volumeto
0,2619.02,2619.88,2678d150-65ad-4cb9-b6ba-47cd042aa6bc,2616.29,2616.29,2017-06-13T00:12:00,3.417760,8957.04
0,2617.55,2617.55,f9a5ae5f-31f3-4213-ac66-ef40152f7bb1,2616.29,2616.29,2017-06-13T00:12:00,0.005262,13.77
0,2619.88,2617.55,288172c7-834b-4b7a-b742-eec54d1027d8,2616.29,2616.29,2017-06-13T00:12:00,0.455960,1193.45
0,2617.56,2619.88,57d3eefb-c032-48f4-bf02-ae7490925274,2616.29,2616.29,2017-06-13T00:12:00,2.742560,7189.41
0,2618.00,2619.02,465f1a93-d22a-46c2-a4b3-7862a2a1cc3c,2618.67,2619.02,2017-06-13T00:15:00,0.004300,11.26


### ThinkDB delete statement

In [389]:
r.db("COIN_PRICES").table("BTCUSD").delete().run()

{'deleted': 33,
 'errors': 0,
 'inserted': 0,
 'replaced': 0,
 'skipped': 0,
 'unchanged': 0}

In [395]:
import time
import datetime
import requests
import rethinkdb as r
import pandas as pd

r.connect( "localhost", 28015).repl()

url = 'https://min-api.cryptocompare.com/data/histohour?fsym=BTC&tsym=USD&limit=10&aggregate=0&e=Kraken'
def data_extract(url):

    while True:
        limit = 10
        url_output = requests.get(url).json()['Data']
        for i in limit:
            url_output[i]['time'] = datetime.datetime.isoformat(datetime.datetime.fromtimestamp(url_output[i]['time']))
#         url_output[1]['time'] = datetime.datetime.isoformat(datetime.datetime.fromtimestamp(url_output[1]['time']))
        #df = pd.DataFrame(url_output)
        #df.time = pd.to_datetime(df.time, unit='s')

        

        #display(df)
            r.db("COIN_PRICES").table("BTCUSD").insert(url_output[i]['time']).run()

        

data_extract(url)  

TypeError: 'int' object is not iterable

In [106]:
# Reading data from coinmarketcap about the historical snapshots of the currencies and their rank and other stats
from IPython.display import display
df = pd.DataFrame(pd.read_html('https://coinmarketcap.com/historical/20170507/')[0])
#df.set_index('#', inplace=True)

In [186]:
# supress scientific notation in pandas by setting format of floats
pd.set_option('display.float_format', lambda x: '%.2f' % x)

# replacing/removing not needed values 
df1 = df.replace({'\$':'', '%':'', '\*':'', '\?': np.nan, 'Low Vol': np.nan, ',':''}, regex = True)

# casting strings  with NaN to floats
df1 = df1.apply(pd.to_numeric, errors='ignore')
display(df1.sort_values(by='Market Cap', ascending=False, na_position='last'))

#df1.Price.describe()

,Name,Symbol,Market Cap,Price,Circulating Supply,Volume (24h),% 1h,% 24h,% 7d
#,,,,,,,,,
1,Bitcoin,BTC,25764555031.00,1579.18,16315180.00,617303973.00,-0.03,1.30,19.82
2,Ethereum,ETH,8794353554.00,96.24,91383653.00,138302702.00,-0.09,2.13,39.05
3,Ripple,XRP,4127097878.00,0.11,37955579225.00,90944104.00,2.31,9.29,108.67
4,Litecoin,LTC,1525287837.00,29.92,50983883.00,142364747.00,0.47,13.51,88.77
5,Dash,DASH,726219134.00,99.74,7281119.00,16904995.00,0.03,0.50,5.48
6,NEM,XEM,665663158.00,0.07,8999999999.00,6163148.00,0.51,0.64,51.06
7,Ethereum Classic,ETC,661203955.00,7.24,91383593.00,30725739.00,0.00,-1.56,30.69
8,Monero,XMR,423240289.00,29.35,14418791.00,14118589.00,0.14,0.63,23.65
9,Golem,GNT,206829437.00,0.25,820000000.00,4569161.00,-0.08,2.45,42.05


count    766.00
mean      23.92
std      281.33
min        0.00
25%        0.00
50%        0.01
75%        0.12
max     6318.08
Name: Price, dtype: float64

In [109]:
r.db("COIN_PRICES").table("BTCUSD").insert([{'close': 235.4,
  'high': 235.45,
  'low': 235.01,
  'open': 235.45,
  'time': 1496839500,
  'volumefrom': 74.81,
  'volumeto': 17606.41},
 {'close': 234.59,
  'high': 235.4,
  'low': 234.59,
  'open': 235.4,
  'time': 1496839680,
  'volumefrom': 38.21,
  'volumeto': 8977.23}]).run()



NameError: name 'Timestamp' is not defined

### MongoDB part

In [175]:
from pymongo import MongoClient

client = MongoClient()

db = client.primer

BTCUSD_PRICES = db.btcusd_prices

BTCUSD_PRICES.insert_many([{'close': 0.4,
  'high': 235.45,
  'low': 235.01,
  'open': 235.45,
  'time': 1496839500,
  'volumefrom': 74.81,
  'volumeto': 17606.41},
 {'close': 1.59,
  'high': 235.4,
  'low': 234.59,
  'open': 235.4,
  'time': 1496839680,
  'volumefrom': 38.21,
  'volumeto': 8977.23}])

In [193]:
df_from_db_m.time.dtype

dtype('O')

In [195]:
df_from_db_m = pd.DataFrame()
for i in BTCUSD_PRICES.find():
    #display(pd.io.json.json_normalize(i))
    df_from_db_m = df_from_db_m.append(pd.io.json.json_normalize(i))
    print(df_from_db_m.describe())
    #df_from_db_m['time'] = pd.to_datetime(df_from_db_m['time'].astype(int), unit="s")
    
df_from_db_m.time.describe()
    

       close    high     low    open          time  volumefrom  volumeto
count    1.0    1.00    1.00    1.00  1.000000e+00        1.00      1.00
mean   235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
std      NaN     NaN     NaN     NaN           NaN         NaN       NaN
min    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
25%    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
50%    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
75%    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
max    235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
       close    high     low    open          time  volumefrom  volumeto
count    2.0    2.00    2.00    2.00  2.000000e+00        2.00      2.00
mean   235.4  235.45  235.01  235.45  1.496840e+09       74.81  17606.41
std      0.0    0.00    0.00    0.00  0.000000e+00        0.00      0.00
min    235.4  235.45  235.01  235.45  1.496840e+09 

count    1.000000e+01
mean     1.496840e+09
std      7.589466e+01
min      1.496840e+09
25%      1.496840e+09
50%      1.496840e+09
75%      1.496840e+09
max      1.496840e+09
Name: time, dtype: float64